## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,75,16.11,light rain
1,1,Nome,US,64.5011,-165.4064,28.47,74,90,9.22,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,48.09,81,38,10.89,scattered clouds
3,3,Souillac,MU,-20.5167,57.5167,73.78,73,75,11.50,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,75.25,83,92,3.44,overcast clouds


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your vacation? "))
max_temp = float(input("What is the maximum temperature you would like for your vacation? "))

What is the minimum temperature you would like for your vacation? 20
What is the maximum temperature you would like for your vacation? 90


In [26]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)] 
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,75,16.11,light rain
1,1,Nome,US,64.5011,-165.4064,28.47,74,90,9.22,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,48.09,81,38,10.89,scattered clouds
3,3,Souillac,MU,-20.5167,57.5167,73.78,73,75,11.50,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,75.25,83,92,3.44,overcast clouds
5,5,Hami,CN,42.8000,93.4500,64.74,36,77,23.13,broken clouds
6,6,Sinop,TR,41.7711,34.8709,52.23,87,61,3.44,broken clouds
7,7,Bambous Virieux,MU,-20.3428,57.7575,73.65,73,75,11.50,broken clouds
8,8,Cabo San Lucas,MX,22.8909,-109.9124,80.91,73,1,0.00,clear sky
9,9,Vaini,TO,-21.2000,-175.2000,71.76,100,40,8.05,scattered clouds


In [27]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                660
City                   660
Country                656
Lat                    660
Lng                    660
Max Temp               660
Humidity               660
Cloudiness             660
Wind Speed             660
Current Description    660
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,42.91,70,75,16.11,light rain
1,1,Nome,US,64.5011,-165.4064,28.47,74,90,9.22,overcast clouds
2,2,Busselton,AU,-33.6500,115.3333,48.09,81,38,10.89,scattered clouds
3,3,Souillac,MU,-20.5167,57.5167,73.78,73,75,11.50,broken clouds
4,4,Avarua,CK,-21.2078,-159.7750,75.25,83,92,3.44,overcast clouds


In [29]:
clean_df.count()

City_ID                656
City                   656
Country                656
Lat                    656
Lng                    656
Max Temp               656
Humidity               656
Cloudiness             656
Wind Speed             656
Current Description    656
dtype: int64

In [30]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Punta Arenas,CL,42.91,light rain,-53.1500,-70.9167,
1,Nome,US,28.47,overcast clouds,64.5011,-165.4064,
2,Busselton,AU,48.09,scattered clouds,-33.6500,115.3333,
3,Souillac,MU,73.78,broken clouds,-20.5167,57.5167,
4,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,
5,Hami,CN,64.74,broken clouds,42.8000,93.4500,
6,Sinop,TR,52.23,broken clouds,41.7711,34.8709,
7,Bambous Virieux,MU,73.65,broken clouds,-20.3428,57.7575,
8,Cabo San Lucas,MX,80.91,clear sky,22.8909,-109.9124,
9,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,


In [31]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [32]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name


In [33]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))